<a href="https://colab.research.google.com/github/statistics-jun/2022-1-ESAA/blob/main/ESAA_0516_TMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2022-1 ESAA 5기 김혜준 실습 필사 - 0516(월) 11주차 과제
 
## <파이썬 머신러닝 완벽 가이드>
## Chapter 9 추천시스템

### 05 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트
* TMDB 5000 영화 데이터 세트 : IMDB의 주요 5000개 영화에 대한 메타 정보를 새롭게 가공해 캐글에서 제공하는 데이터 세트

#### 장르 속성을 이용한 영화 콘텐츠 기반 필터링
* 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성/속성, 구성 요소 등을 가진 다른 영화를 추천
* 영화 간의 유사성을 판단하는 기준이 영화를 구성하는 다양한 콘텐츠(장르, 감독, 배우, 평점, 키워드, 영화 설명)를 기반으로 하는 방식
* 영화 장르 속성 기반 -> 장르 속성 칼럼 값의 유사도를 비교한 뒤 그중 높은 평점을 가지는 영화를 추천하는 방식

#### 데이터 로딩 및 가공

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### 데이터 로딩

In [2]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv('/content/drive/MyDrive/ESAA/2022-1/과제/실습 필사/0516(월) 11주차 과제/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


> 4803개의 레코드, 20개의 피처로 구성
  - 영화 제목, 개요, 인기도, 평점, 투표 수, 예산, 키워드 등 영화에 대한 메타 정보

##### 주요 칼럼만 추출해 새로운 DataFrame 생성

In [3]:
movies_df = movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity',
                    'keywords', 'overview']]

* 'genres', 'keywords' 칼럼 : 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열
  
  -> DataFrame으로 만들어질 때 단순히 문자열 형태로 로딩되므로 필요한 정보 추출 위한 가공 필요 

In [4]:
# genres, keywords 칼럼 형태 확인
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


* genres 칼럼 : 여러 개의 개별 장르 데이터로 구성

##### genres 칼럼의 문자열을 분해해서 개별 장르를 파이썬 리스트 객체로 추출

In [5]:
from ast import literal_eval # 문자열을 문자열이 의미하는 list[ dict1, dict2 ] 객체로 만드는 함수
movies_df['genres'] = movies_df['genres'].apply(literal_eval) # 문자열을 객체로 변환
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

* genres 칼럼이 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체로 변환

In [6]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x]) # 리스트 내 여러 개의 딕셔너리의 'name' 키에 해당하는 값(장르명)만 리스트 객체로 추출
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


#### 장르 콘텐츠 유사도 측정

* genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링 구현 단계
  1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환
  2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도로 비교 : 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체 생성
  3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화 추천

##### genres 칼럼을 문자열로 변환하고 사이킷런의 CountVectorizer로 피처 벡터 행렬 생성

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x)) # 개별 요소를 공백 문자로 구분하는 문자열로 변환
count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


> 4803개의 레코드, 276개의 개별 단어 피처로 구성

##### 피처 벡터 행렬에 사이킷런의 cosine_similarity() 함수 이용해 코사인 유사도 계산

In [8]:
from sklearn.metrics.pairwise import cosine_similarity # 기준 행과 비교 행의 코사인 유사도를 행렬 형태로 반환하는 함수

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


* genre_sim 객체 : movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬(genre_mat) 데이터의 행(레코드)별 유사도 정보
  
  => movie_df의 행별 장르 유사도 값

##### genre_sim 객체의 기준 행별로 비교 대상이 되는 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값 추출

In [9]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


> 0번 레코드 : 자신을 제외하고 3494번 레코드가 가장 유사도가 높고, 2401번 레코드가 가장 유사도가 낮다고 해석

* genre_sim_sorted_ind : 각 레코드의 장르 코사인 유사도가 높은 순으로 정렬된 타 레코드의 위치 인덱스 값
  
  -> 특정 레코드와 코사인 유사도가 높은 다른 레코드 추출 가능

#### 장르 콘텐츠 필터링을 이용한 영화 추천

##### 장르 유사도에 따라 영화를 추천하는 함수 생성

In [10]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  '''
  df : 기반 데이터 movies_df
  sorted_ind : 레코드별 장르 코사인 유사도 인덱스 genre_sim_sorted_ind
  title_name : 고객이 선정한 추천 기준이 되는 영화 제목
  top_n : 추천할 영화 건수
  '''
  # 인자로 입력된 movies_df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
  title_movie = df[df['title'] == title_name]

  # title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index, :(top_n)]

  # 추출된 top_n index 출력. top_n index는 2차원 데이터임
  # DataFrame에서 index로 사용하기 위해서 1차원 array로 변경
  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes] # 추천 영화 정보를 가지는 DataFrame 반환

In [11]:
# 영화 '대부'와 장르별로 유사한 영화 10개 추천
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


> 대부 2편이 가장 먼저 추천
> 
> 그러나 낯선 영화도 많음

* vote_average : 영화의 평점 정보, 0점~10점, 여러 관객이 평가한 평점의 평균
  - 소수 관객이 특정 영화에 높은 평점을 부여해 왜곡된 데이터가 존재

In [12]:
# vote_average 상위 10개 확인
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


> 유명하지 않은 영화가 더 높은 평점을 가짐
> 
> -> 평가 횟수가 매우 작음

* 왜곡된 평점 데이터 회피하도록 평점에 평가 횟수에 대한 가중치가 부여된 '가중 평점' 방식 도입
  - 가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/v+m)) * C
    - v : 개별 영화에 평점을 투표한 횟수 (`movie_df`의 `vote_count`)
    - m : 평점을 부여하기 위한 최소 투표 횟수
      - 투표 횟수에 따른 가중치를 직접 조절하는 역할 수행
      - m 값을 높이면 평점 투표 횟수가 많은 영화에 더 많은 가중 평점 부여
    - R : 개별 영화에 대한 평균 평점 (`movie_df`의 `vote_average`)
    - C : 전체 영화에 대한 평균 평점 (`movie_df[vote_average].mean()`)

In [13]:
C = movies_df['vote_average'].mean() # 전체 영화에 대한 평균 평점
m = movies_df['vote_count'].quantile(0.6) # 전체 투표 횟수 중 상위 60%에 해당하는 횟수가 기준
print('C:', round(C, 3), 'm:', round(m, 3))

C: 6.092 m: 370.2


##### 기존 평점을 새로운 가중 평점으로 변경하는 함수 생성

In [14]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
  '''
  record : DataFrame의 레코드
  '''
  v = record['vote_count']
  R = record['vote_average']

  return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C ) # 레코드별 가중 평점 반환

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) # 가중 평점 계산

In [15]:
# weighted_vote 평점 상위 10개 영화 추출
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


> 뛰어난 영화가 Top 10에 선정

##### 새롭게 정의된 가중 평점에 따라 영화를 추천하도록 함수 변경

In [16]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie = df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  similar_indexes = similar_indexes.reshape(-1)
  # 기준 영화 인덱스는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


> 이전보다 훨씬 나은 영화가 추천됨